## 使用python部署so文件

### 1.导包

In [1]:
# 使用python部署*.so文件
import onnx
import time
import tvm
import numpy as np
import tvm.relay as relay
from PIL import Image
from tvm.contrib import graph_runtime



### 2.Load a test image

In [2]:
# ------------ load image
# img = Image.open("street.jpg")
img = Image.open("test.png")
img = img.resize((416, 416))
img = np.array(img, dtype = np.float32)
img /= 255.0
img = img.transpose((2, 0, 1))
img = np.expand_dims(img, axis=0)

### 3.load model

In [3]:

libpath = "../models/yolov4_pc.so"
graph_json_path = "../models/yolov4_pc.json"
param_path = "../models/yolov4_pc.params"

# 接下来我们加载导出的模型去测试导出的模型是否可以正常工作
loaded_json = open(graph_json_path).read()
loaded_lib = tvm.runtime.load_module(libpath)
loaded_params = bytearray(open(param_path, "rb").read())

# 这里执行的平台为CPU
ctx = tvm.cpu()


module = graph_runtime.create(loaded_json, loaded_lib, ctx)
module.load_params(loaded_params)
module.set_input("input_0", img)
module.run()
out_deploy = module.get_output(0).asnumpy()
print(type(out_deploy))
print(out_deploy[0][0][0])


<class 'numpy.ndarray'>
[ 1.7110772   0.10599595  0.12194884 -0.42449266 -1.4188616   0.64269596
  0.32359695 -0.0385932  -1.0393591   0.37409925  0.1668087   0.81054145
 -2.1574116 ]


In [6]:
# 输出tvm运行的时间
since = time.time()
for i in range(100):
    module.run()
time_elapsed = time.time() - since
print('Time elapsed is {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))

Time elapsed is 0m 38s
